In [ ]:
#@title 💽 Install RVC & Connect to Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from multiprocessing import cpu_count
cpu_cores = cpu_count()
LOGS_PATH = "/content/Applio/logs"

%cd /content
!git config --global advice.detachedHead false
!git clone https://github.com/IAHispano/Applio --branch 3.5.1 --single-branch
%cd /content/Applio

!apt update -y
!apt install -y python3.11 python3.11-distutils python3.11-dev portaudio19-dev
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!update-alternatives --set python3 /usr/bin/python3.11
from sys import path
path.append('/usr/local/lib/python3.11/dist-packages')

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match
!uv pip install -q jupyter-ui-poll
!uv pip install -q "audio-separator[gpu]==0.17.5"
!uv pip install -q demucs yt_dlp pydub
!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"
print("Finished installing requirements!")

In [ ]:
#@title 🎵✂️🎤 Separation (BS-Roformer and Mel Band Roformer)
#@markdown #### در صورتی که تراک را بصورت دستی آپلود کردین، فیلد اول را به همین شکل باقی بگذارید
import os
import glob
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': '/content/temp/%(title)s.%(ext)s',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model, overlap, segment_size):
    found_files = []
    dictmodel = {
        'BS-Roformer-Viperx-1297.ckpt': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
        'BS-Roformer-Viperx-1296.ckpt': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
        'BS-Roformer-Viperx-1053.ckpt': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
        'Mel-Roformer-Viperx-1143.ckpt': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt'
    }
    roformer_model = dictmodel[model]

    if checker(audio_input):
        os.makedirs('/content/temp', exist_ok=True)
        downloader(audio_input)
        audio_input = "/content/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)
    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")
        found_files.sort()
        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {roformer_model} --output_dir={output_folder} --output_format={output_format} --normalization=0.9 --mdxc_overlap={overlap} --mdxc_segment_size={segment_size}'
            !$prompt

    if audio_input == "/content/temp":
        temp_files = glob.glob("/content/temp/*")
        for file in temp_files:
            os.remove(file)

audio_input = "/content" #@param {type:"string"}
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
model = "BS-Roformer-Viperx-1297.ckpt" #@param ["BS-Roformer-Viperx-1297.ckpt", "BS-Roformer-Viperx-1296.ckpt", "BS-Roformer-Viperx-1053.ckpt", "Mel-Roformer-Viperx-1143.ckpt"]
output_format = "wav" #@param ["wav", "flac", "mp3"]
overlap = 4 #@param {type:"slider", min:2, max:4, step:1}
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
extensions = (".mp3", ".wav", ".flac")

os.makedirs(output_folder, exist_ok=True)
uvr_cli(audio_input, output_folder, extensions, output_format, model, overlap, segment_size)

In [ ]:
#@title Load Model (ZIP, TAR.GZ, or Drive Link)
Model_Name = "" #@param {type:"string"}
MODEL_LINK = "" #@param {type:"string"}

import gdown
import os
import glob
import shutil
import tarfile  # Import tarfile for signature check

if not Model_Name:
    print("❌ Error: Please enter the Model Name.")
else:
    # Create model directory in target path
    target_dir = f"/content/Applio/logs/{Model_Name}"
    os.makedirs(target_dir, exist_ok=True)
    print(f"Target directory created/exists: {target_dir}")

    # --- Download ---
    downloaded_file_path = "/content/downloaded_model"
    try:
        if 'drive.google.com' in MODEL_LINK:
            print("Downloading from Google Drive...")
            gdown.download(MODEL_LINK, downloaded_file_path, quiet=False, fuzzy=True)
        elif MODEL_LINK.endswith(('.zip', '.tar.gz')):
            print(f"Downloading from direct link ({MODEL_LINK})...")
            os.system(f'wget -O {downloaded_file_path} "{MODEL_LINK}"')
        else:
            print("❌ Error: Unsupported link type. Please provide a Google Drive link or a direct link to a .zip or .tar.gz file.")
            downloaded_file_path = None  # Indicate download failure

        if downloaded_file_path and os.path.exists(downloaded_file_path):
            print("✅ Download successful.")
        elif downloaded_file_path:
            print("❌ Error: Download failed or file not found after download attempt.")
            downloaded_file_path = None

    except Exception as e:
        print(f"❌ Error during download: {e}")
        downloaded_file_path = None

    # --- Extraction ---
    if downloaded_file_path:
        file_type = None
        try:
            # Try identifying as tar.gz first
            with tarfile.open(downloaded_file_path, 'r:*') as tar:  # auto-detect compression
                file_type = 'tar'
                print("Detected file type: tar archive")
                temp_extract_dir = "/content/temp_extract"
                os.makedirs(temp_extract_dir, exist_ok=True)
                print(f"Extracting {downloaded_file_path} to {temp_extract_dir}...")
                tar.extractall(path=temp_extract_dir)

        except tarfile.ReadError:
            # If not a tarfile, try as zip
            try:
                import zipfile
                if zipfile.is_zipfile(downloaded_file_path):
                    file_type = 'zip'
                    print("Detected file type: zip archive")
                    print(f"Extracting {downloaded_file_path} to {target_dir}...")
                    with zipfile.ZipFile(downloaded_file_path, 'r') as zip_ref:
                        zip_ref.extractall(target_dir)
                else:
                    print("❌ Error: File is not a valid zip or tar archive.")
            except Exception as zip_e:
                print(f"❌ Error processing as zip file: {zip_e}")

        except Exception as e:
            print(f"❌ Error during extraction detection/setup: {e}")

        # --- Copying/Moving files specifically for TAR archives ---
        if file_type == 'tar':
            files_copied = False
            # Search and copy required files using os.walk
            for root, dirs, files in os.walk(temp_extract_dir):
                for file in files:
                    if file.endswith('.pth'):
                        source_path = os.path.join(root, file)
                        dest_path = os.path.join(target_dir, file)
                        shutil.copy2(source_path, dest_path)
                        print(f"Copied PTH: {file} to {target_dir}")
                        files_copied = True

                    if file.endswith('.index'):
                        source_path = os.path.join(root, file)
                        dest_path = os.path.join(target_dir, file)
                        shutil.copy2(source_path, dest_path)
                        print(f"Copied INDEX: {file} to {target_dir}")
                        files_copied = True

            if not files_copied:
                print("⚠️ Warning: No .pth or .index files found during deep search in extracted tar.")

            # Clean up temp extraction dir for tar
            if os.path.exists(temp_extract_dir):
                shutil.rmtree(temp_extract_dir)
                print(f"Removed temporary extraction directory: {temp_extract_dir}")

        # --- Cleanup Extraneous PTH Files (for BOTH zip and tar) ---
        print("\n🧹 Starting cleanup of extraneous PTH files...")
        expected_model_pth_filename = f"{Model_Name}.pth"
        expected_model_pth_full_path = os.path.join(target_dir, expected_model_pth_filename)

        # List all .pth files currently in the target directory
        pth_files_in_target = glob.glob(os.path.join(target_dir, "*.pth"))
        found_expected_pth = False

        if not pth_files_in_target:
            print(f"❌ Warning: No .pth files found in the final target directory {target_dir}.")
        else:
            print(f"Found PTH files: {[os.path.basename(f) for f in pth_files_in_target]}")
            # Check if the main expected file exists
            if expected_model_pth_full_path in pth_files_in_target:
                found_expected_pth = True
                print(f"✅ Expected model file '{expected_model_pth_filename}' found.")
            else:
                print(f"⚠️ Warning: Expected model file '{expected_model_pth_filename}' NOT found.")

            # Remove files starting with G_ or D_ OR any pth file if the expected one exists but this one isn't it
            for pth_file_path in pth_files_in_target:
                filename = os.path.basename(pth_file_path)
                if filename.startswith("G_") or filename.startswith("D_"):
                    try:
                        os.remove(pth_file_path)
                        print(f"🧹 Removed checkpoint file: {filename}")
                    except OSError as e:
                        print(f"❌ Error removing file {filename}: {e}")
                elif found_expected_pth and pth_file_path != expected_model_pth_full_path:
                    try:
                        os.remove(pth_file_path)
                        print(f"🧹 Removed other PTH file (since expected exists): {filename}")
                    except OSError as e:
                        print(f"❌ Error removing file {filename}: {e}")

        # 🔍 اینجا منطق پیشنهادی نام صحیح مدل را اضافه می‌کنیم
        final_pth_files = glob.glob(os.path.join(target_dir, "*.pth"))
        if final_pth_files:
            # اگر نام وارد شده باعث نشده فایل {Model_Name}.pth ساخته شود،
            # ولی یک یا چند pth دیگر هست، یکی را پیشنهاد بده
            if expected_model_pth_full_path not in final_pth_files:
                # ساده‌ترین حالت: از آخرین pth به‌عنوان نام صحیح استفاده کن
                suggested_full = sorted(final_pth_files)[-1]
                suggested_base = os.path.basename(suggested_full)
                suggested_name = os.path.splitext(suggested_base)[0]

                print("\n⚠️ توجه: به نظر می‌رسد نامی که برای مدل وارد کردید با نام واقعی فایل pth یکی نیست.")
                print(f"نام صحیح مدل بر اساس فایل موجود احتمالاً این است:")
                print(f"👉  {suggested_name}")
                print("لطفاً همین نام را بدون پسوند در فیلد نام مدل وارد کنید تا سلول اینفرِنس درست کار کند.")

        # Final listing to confirm
        print(f"\n✅ Extraction and cleanup complete for model '{Model_Name}'. Final contents:")
        os.system(f'ls -la "{target_dir}"')

        # Clean up the downloaded archive file
        os.remove(downloaded_file_path)
        print(f"Removed downloaded archive: {downloaded_file_path}")

    else:
        print("Skipping extraction due to download failure.")


In [ ]:
#@title Run Inference (Auto Load Vocals from Drive)
%cd /content/Applio
from pathlib import Path
import os
import shutil

input_directory = "/content/drive/MyDrive/Vocales"

audio_file = None
for file in os.listdir(input_directory):
    if "Vocals" in file and file.lower().endswith(('.mp3', '.wav', '.flac', '.ogg', '.m4a')):
        audio_file = os.path.join(input_directory, file)
        break

if audio_file is None:
    print("No Vocals file found!")
    raise SystemExit

new_audio_path = "/content/input.wav"
shutil.copy(audio_file, new_audio_path)
print(f"Loaded: {audio_file}")

model_name = "" #@param {type:"string"}
model_path = Path(f"{LOGS_PATH}/{model_name}")
if not (model_path.exists() and model_path.is_dir()):
    raise FileNotFoundError(f"Model directory not found: {model_path.resolve()}")

!ls -t "{model_path}"/"{model_name}"_*e_*s.pth "{model_path}"/"{model_name}".pth 2> /dev/null | head -n 1 > /tmp/pth.txt
pth_file = open("/tmp/pth.txt", "r").read().strip()
if pth_file == "":
    raise FileNotFoundError(f"No model weight found")

!ls -t "{model_path}"/*.index | head -n 1 > /tmp/index.txt
index_file = open("/tmp/index.txt", "r").read().strip()

input_path = new_audio_path
output_path = "/content/output.wav"
export_format = "WAV" #@param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A']
f0_method = "rmvpe" #@param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"]
f0_up_key = 0 #@param {type:"slider", min:-24, max:24, step:1}
rms_mix_rate = 0.8 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5 #@param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
clean_strength = 0.7 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False #@param{type:"boolean"}
clean_audio = False #@param{type:"boolean"}
f0_autotune = False #@param{type:"boolean"}
formant_shift = False #@param{type:"boolean"}
formant_qfrency = 1.0 #@param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 #@param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" #@param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"]
embedder_model_custom = "" #@param {type:"string"}

!rm -f "{output_path}"
!python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}"

if Path(output_path).exists():
    from IPython.display import Audio, display
    output_path = output_path.replace(".wav", f".{export_format.lower()}")
    display(Audio(output_path, autoplay=True))

In [ ]:
#@title Mix with Instrumental
#@markdown ####  🔀 ترکیب صدای تبدیل‌شده با موزیک پس‌زمینه
import os
from pydub import AudioSegment

music_directory = "/content/drive/MyDrive/Vocales"

music_file = None
for file in os.listdir(music_directory):
    if "Instrumental" in file and file.lower().endswith(('.mp3', '.wav', '.flac', '.ogg', '.m4a')):
        music_file = os.path.join(music_directory, file)
        break

if music_file is None:
    print("No Instrumental file found!")
    raise SystemExit

processed_audio_path = "/content/output.wav"
if not os.path.exists(processed_audio_path):
    print("Processed audio not found!")
    raise SystemExit

voice = AudioSegment.from_file(processed_audio_path)
music = AudioSegment.from_file(music_file)

if len(music) > len(voice):
    music = music[:len(voice)]
else:
    voice = voice[:len(music)]

mixed_audio = voice.overlay(music, position=0, gain_during_overlay=-5)
final_output_path = "/content/mixed_output.wav"
mixed_audio.export(final_output_path, format="wav")

from IPython.display import Audio, display
display(Audio(final_output_path, autoplay=True))

In [ ]:
#@title Download Final Output
#@markdown ####📥 دانلود فایل نهایی میکس‌ شده
from google.colab import files
import glob
import os
from IPython.display import Audio, display

output_files = glob.glob("/content/output.*") + glob.glob("/content/mixed_output.*")
if not output_files:
    print("No output files found")
else:
    latest_file = max(output_files, key=os.path.getmtime)
    print(f"Downloading: {os.path.basename(latest_file)}")
    display(Audio(latest_file, autoplay=False))
    files.download(latest_file)